In [1]:
from tools import *
import numpy as np

In [2]:
PATH_TO_LVK_DATA = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_analysis/GWTC_LVK_data/'
PATH_TO_RESULTS = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_analysis/'
PATH_TO_YAML = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_analysis/yaml/'
PATH_TO_PSD = '/Users/ulyana/Documents/GSSI/PhD Projects/GWTC_analysis/gwtc_psds/'

In [3]:
waveform = 'IMRPhenomXPHM'

In [4]:
params = ['chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 
            'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl']

In [5]:
events = np.loadtxt(PATH_TO_RESULTS + 'info/' + 'events_with_priors_%s.txt' %waveform, dtype=str)
with open(PATH_TO_RESULTS + 'info/' + 'detectors_dictionary.pkl', 'rb') as f:
    detectors = pickle.load(f)

In [6]:
for event in events:

    detectors_list = detectors[event]
    detectors_event = []
    for j in range(len(detectors_list)):
        detectors_event.append(detectors_list[j])
    networks = np.linspace(0, len(detectors_event) - 1, len(detectors_event), dtype=int)
    networks = str([networks.tolist()])

    detectors_ids = np.array(detectors_event)
    networks_ids = json.loads(networks)
    ConfigDet = os.path.join(PATH_TO_YAML + event + '.yaml')


    waveform_model = waveform
    gw_parameters_median = pd.read_hdf(PATH_TO_RESULTS + 'injections/' + event +  '/%s_%s_median.hdf5' %(event, waveform))
    gw_parameters_maxP = pd.read_hdf(PATH_TO_RESULTS + 'injections/' + event +  '/%s_%s_maxP.hdf5' %(event, waveform))

    gw_parameters_median = gw_parameters_median[params]
    gw_parameters_maxP = gw_parameters_maxP[params]

    line_parametrization = np.linspace(0, 1, 10)

    for i in range(len(line_parametrization)):
        gw_parameters = (1 - line_parametrization[i]) * gw_parameters_maxP + line_parametrization[i] * gw_parameters_median

        population_label = 'from_median_to_maxP_%s_BBH_%s' %(i, event)

        network = gw.detection.Network(detectors_ids, detection_SNR=(0., 1.), config=ConfigDet)
        gw.fishermatrix.analyze_and_save_to_txt(network = network,
                                        parameter_values  = gw_parameters,
                                        fisher_parameters = params, 
                                        sub_network_ids_list = networks_ids,
                                        population_name = population_label,
                                        waveform_model = waveform_model,
                                        save_path = PATH_TO_RESULTS + 'results/gwfish_from_median_to_maxP/' + event,
                                        save_matrices = True,
                                        decimal_output_format='%.15f')

100%|██████████| 1/1 [00:19<00:00, 19.15s/it]
